# Домашнее задание к вебинару 9

Задание: обучить генератор текстов
* В этом ноутбуке попробуем генерировать текст не по символам, а по токенам.

In [462]:
pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [463]:
import tensorflow as tf

import numpy as np
import os
import time

from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords

from string import punctuation
from stop_words import get_stop_words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [464]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [465]:
path_to_file = '/content/evgenyi_onegin.txt'

In [466]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [467]:
# Выведем фрагмент текста
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [468]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [469]:
sw = get_stop_words("ru") + stopwords.words('russian')
punkt = list(
    punctuation)
noise = set(sw + punkt)

In [470]:
# токенизация текста 

def make_tokens(text):
    return [token for token in word_tokenize(text) if token not in noise and token.isalpha()]
tokens = make_tokens(text)

In [471]:
# Создадим словарь уникальных токенов в тексте
vocab = sorted(set(tokens))
print('{} unique characters'.format(len(vocab)))

8911 unique characters


In [472]:
vocab[-10:]

['яснее',
 'яснеет',
 'ясно',
 'ясной',
 'ясностию',
 'ясностью',
 'ясною',
 'ясные',
 'ясным',
 'ящик']

In [473]:
# Задаем отображения из множества уникальных букв в множество индексов
token2idx = {u:i for i, u in enumerate(vocab)} # словарь {индекс: токен}

idx2token = np.array(vocab) # array из символов словаря (можно по индексу извлекать токен)

In [474]:
idx2token

array(['Annеttе', 'Au', 'Byron', ..., 'ясные', 'ясным', 'ящик'],
      dtype='<U18')

In [475]:
# Теперь переводим все символы текста в индексы - на выходе array из индексов
text_as_int = np.array([token2idx[c] for c in tokens])

In [476]:
text_as_int

array([ 153, 1743, 1609, ..., 1213, 4921,  808])

In [477]:
print(text[:500]), print(text_as_int[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо
[ 153 1743 1609  582 1209 1666 7751 1062 5046 3223 7185 3800  338 3616
 2871 2105 6487  649 3571  547 3660 6510 1726 4224 4840 1470 3763 8904
  409 3634 6733 1135 7106 6623 1528 7563 5928 3129 1424 6243 1576 4124
 1065 6025 4922 3798  194 4564 2601 1087 8242 2030 8549 5055  708 7299
 3229 3897  415 1283  708 3779 8105 8711 7652  794  414    7  988 3676
 7159 8672 6436  803 8827 3920 1207 8223 3966  708 3063  587 6589 3609
 5159 1135 2458 7381 1685 2476 7322 5413 1062 5774 8749 6753  764 5360
 4

(None, None)

### Подготовка данных к обучению

In [478]:
# Максимальная длина предложения (максимальное количество слов в батче) для входных данных (в буквах)
tokens_length = 200

# Количество эпох
examples_per_epoch = len(text)//(tokens_length+1)

# создаем экземпляр датасета из идексов токенов
tokens_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in tokens_dataset.take(5):
    print(idx2token[i.numpy()], len(tokens_dataset)) 

Александр 17362
Сергеевич 17362
Пушкин 17362
Евгений 17362
Онегин 17362


In [479]:
# Создаем последовательность из батчей токенов с установленной длиной батча
sequences = tokens_dataset.batch(tokens_length+1, drop_remainder = True)

In [480]:
len(sequences), len(tokens), len(tokens_dataset)

(86, 17362, 17362)

In [481]:
for seq in sequences:
  print(seq)
  break

tf.Tensor(
[ 153 1743 1609  582 1209 1666 7751 1062 5046 3223 7185 3800  338 3616
 2871 2105 6487  649 3571  547 3660 6510 1726 4224 4840 1470 3763 8904
  409 3634 6733 1135 7106 6623 1528 7563 5928 3129 1424 6243 1576 4124
 1065 6025 4922 3798  194 4564 2601 1087 8242 2030 8549 5055  708 7299
 3229 3897  415 1283  708 3779 8105 8711 7652  794  414    7  988 3676
 7159 8672 6436  803 8827 3920 1207 8223 3966  708 3063  587 6589 3609
 5159 1135 2458 7381 1685 2476 7322 5413 1062 5774 8749 6753  764 5360
 4369 1421 3802  594 6129 6288 1331 6102 4576  318 3641  803 8660 2881
    8 1903 3638 4936 6057  890 6810 6408  389 2886  692 1049 7054  566
  927 1680 1685 3117 4916 7077  181 6470 7259 8622 1395 6168 1209 3492
 6645 1659 2512 1076  422 7047  734 2429 8691 1908 5289 3337 1135 2992
 7249    9 1789 5739 2383  537 3772 5723  487 2249 3685  708 6696 1888
  584 8573 1840   25 8528 1458   27 7490 1646 7007 4845   27 8485 8216
 2151 4164 3471 2062 8829 1062 3513 4962 7837 1779 8756 2498  708 

In [482]:
for item in sequences.take(5):
    print(repr(' '.join(idx2token[item.numpy()])))

'Александр Сергеевич Пушкин Евгений Онегин Роман стихах Не мысля гордый свет забавить Вниманье дружбы возлюбя Хотел представить Залог достойнее Достойнее души прекрасной Святой исполненной мечты Поэзии живой ясной Высоких дум простоты Но рукой пристрастной Прими собранье пестрых глав Полусмешных полупечальных Простонародных идеальных Небрежный плод моих забав Бессонниц легких вдохновений Незрелых увядших Ума холодных наблюдений И сердца горестных замет ГЛАВА ПЕРВАЯ И жить торопится чувствовать спешит Кн Вяземский I Мой дядя самых честных правил Когда шутку занемог Он уважать заставил И выдумать Его пример другим наука Но боже скука С больным сидеть ночь Не отходя шагу прочь Какое низкое коварство Полуживого забавлять Ему подушки поправлять Печально подносить лекарство Вздыхать думать Когда черт возьмет II Так думал молодой повеса Летя пыли почтовых Всевышней волею Зевеса Наследник родных Друзья Людмилы Руслана С героем моего романа Без предисловий сей час Позвольте познакомить Онегин д

In [483]:
# Разбиваем каждый батч на признаки и целевую переменную (следующую букву)
def split_input_target(batch):
    input_text = batch[:-1]
    target_text = batch[1:]
    return input_text, target_text

#применяем функцию split_input_target ко всем батчам -> таким образом формируем новый датасет
dataset = sequences.map(split_input_target)

In [484]:
for input_example, target_example in dataset:
    print(f'Количество батчей: {len(dataset)}')
    print(f'Данные: {input_example}')
    print(f'Целевая переменная: {target_example}')
    break

Количество батчей: 86
Данные: [ 153 1743 1609  582 1209 1666 7751 1062 5046 3223 7185 3800  338 3616
 2871 2105 6487  649 3571  547 3660 6510 1726 4224 4840 1470 3763 8904
  409 3634 6733 1135 7106 6623 1528 7563 5928 3129 1424 6243 1576 4124
 1065 6025 4922 3798  194 4564 2601 1087 8242 2030 8549 5055  708 7299
 3229 3897  415 1283  708 3779 8105 8711 7652  794  414    7  988 3676
 7159 8672 6436  803 8827 3920 1207 8223 3966  708 3063  587 6589 3609
 5159 1135 2458 7381 1685 2476 7322 5413 1062 5774 8749 6753  764 5360
 4369 1421 3802  594 6129 6288 1331 6102 4576  318 3641  803 8660 2881
    8 1903 3638 4936 6057  890 6810 6408  389 2886  692 1049 7054  566
  927 1680 1685 3117 4916 7077  181 6470 7259 8622 1395 6168 1209 3492
 6645 1659 2512 1076  422 7047  734 2429 8691 1908 5289 3337 1135 2992
 7249    9 1789 5739 2383  537 3772 5723  487 2249 3685  708 6696 1888
  584 8573 1840   25 8528 1458   27 7490 1646 7007 4845   27 8485 8216
 2151 4164 3471 2062 8829 1062 3513 4962 7837 1

In [485]:
for input_example, target_example in dataset.take(1):
    print('Input data: ', repr(' '.join(idx2token[input_example.numpy()])))
    print('Target data:', repr(' '.join(idx2token[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин Евгений Онегин Роман стихах Не мысля гордый свет забавить Вниманье дружбы возлюбя Хотел представить Залог достойнее Достойнее души прекрасной Святой исполненной мечты Поэзии живой ясной Высоких дум простоты Но рукой пристрастной Прими собранье пестрых глав Полусмешных полупечальных Простонародных идеальных Небрежный плод моих забав Бессонниц легких вдохновений Незрелых увядших Ума холодных наблюдений И сердца горестных замет ГЛАВА ПЕРВАЯ И жить торопится чувствовать спешит Кн Вяземский I Мой дядя самых честных правил Когда шутку занемог Он уважать заставил И выдумать Его пример другим наука Но боже скука С больным сидеть ночь Не отходя шагу прочь Какое низкое коварство Полуживого забавлять Ему подушки поправлять Печально подносить лекарство Вздыхать думать Когда черт возьмет II Так думал молодой повеса Летя пыли почтовых Всевышней волею Зевеса Наследник родных Друзья Людмилы Руслана С героем моего романа Без предисловий сей час Позвольте познако

### Построение модели

In [486]:
# размер батча
BATCH_SIZE = 16

# размер буфера для перемешивания данных
BUFFER_SIZE = 10000

# перемешивание разделенных данных
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [487]:
len(dataset)

5

In [488]:
# Длина словаря
vocab_size = len(vocab)

# Длина выходного эмбеддинга
embedding_dim = 512

# Количество скрытых состояний в RNN слое 
rnn_units = 1024

In [489]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [490]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [491]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (16, None, 512)           4562432   
                                                                 
 lstm_47 (LSTM)              (16, None, 1024)          6295552   
                                                                 
 lstm_48 (LSTM)              (16, None, 1024)          8392704   
                                                                 
 lstm_49 (LSTM)              (16, None, 1024)          8392704   
                                                                 
 lstm_50 (LSTM)              (16, None, 1024)          8392704   
                                                                 
 dense_12 (Dense)            (16, None, 8911)          9133775   
                                                                 
Total params: 45,169,871
Trainable params: 45,169,871

**Сделаем предсказание без обучения модели**

In [492]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape," # (batch_size, sequence_length, vocab_size)")

(16, 200, 8911)  # (batch_size, sequence_length, vocab_size)


In [493]:
example_batch_predictions[0]

<tf.Tensor: shape=(200, 8911), dtype=float32, numpy=
array([[-2.16110675e-06,  5.02403191e-06, -5.07310779e-06, ...,
        -7.71862233e-06,  4.34601361e-06,  7.25143479e-07],
       [-5.76603679e-06,  1.31537136e-05, -1.53526271e-05, ...,
        -1.60427535e-05,  7.12364590e-06, -6.58121098e-06],
       [-1.73401659e-05,  2.50995981e-05, -3.42587991e-05, ...,
        -2.06162458e-05,  1.82816837e-06, -2.45668871e-05],
       ...,
       [ 3.52714414e-04, -7.33937195e-04,  9.89198452e-04, ...,
        -1.06927146e-04, -7.35093257e-04, -4.51122323e-05],
       [ 3.26732261e-04, -7.33984110e-04,  1.10001664e-03, ...,
        -1.56889160e-04, -7.79844820e-04,  2.60967113e-06],
       [ 2.75660976e-04, -7.07440719e-04,  1.19228906e-03, ...,
        -2.09166436e-04, -8.00655223e-04,  4.99618400e-05]], dtype=float32)>

In [494]:
# сэмплируем предсказание 
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1) #количество независимых выборок 1

#убираем лишнюю размерность (список индексов)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([8194, 5546, 8726, 1113, 6874, 7619, 3815, 8057, 8776, 5085, 4726,
       2121, 5310, 5850, 7525, 2805, 8686, 4963, 2254, 5166,  635, 1313,
       3142, 1904, 8118, 2654, 7635, 5479, 4905, 6062, 8327, 3316, 2730,
       2715, 4426, 1719, 1338, 2108, 1149,  419, 8385, 7111, 5161, 5617,
       5694,  143, 1294, 1246, 1389,  834, 5092, 7743, 6963, 3010, 5993,
       5735,  938, 5063, 2698, 2124, 5788, 4227, 4242, 1562, 1956, 5909,
       4722, 4926, 4699, 1599, 3203, 1005, 6714, 1402, 7768, 6564, 3034,
       7215, 5478, 5219, 5071, 7157, 1996, 7491, 1648, 3906, 2921, 8132,
       4395, 1461, 5433, 2490, 1373, 5284, 5577, 6293, 2129, 6753, 4607,
       6565, 5519, 5270, 7883,  885, 3437, 1246, 4758, 1090, 1567, 8385,
       8510, 2902, 1158, 2150, 3837, 2901, 6550, 3530, 7644,   53, 1916,
       1779, 1549, 5763, 6633, 1395, 4588, 4778, 8827, 3406,  976, 1347,
       4251,  628, 8610, 3329, 3823,  629, 7940, 2670, 3144,  283, 6185,
       3823,  934, 3750, 8790, 7925, 4596, 1175,  6

In [495]:
print("Input: \n", repr(" ".join(idx2token[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr(" ".join(idx2token[sampled_indices ])))

Input: 
 'пень Становится Гильо смущенный Плащи бросают врага Зарецкий тридцать шага Отмерил точностью отменной Друзей развел крайний след И взял свой пистолет XXX Теперь сходитесь Хладнокровно Еще целя врага Походкой твердой тихо ровно Четыре перешли шага Четыре смертные ступени Свой пистолет Евгений Не преставая наступать Стал тихо подымать Вот шагов ступили И Ленский жмуря левый глаз Стал целить Онегин выстрелил Пробили Часы урочные поэт Роняет молча пистолет XXXI На грудь кладет тихонько руку И падает Туманный взор Изображает смерть муку Так медленно скату гор На солнце искрами блистая Спадает глыба снеговая Мгновенным холодом облит Онегин юноше спешит Глядит зовет напрасно Его Младой певец Нашел безвременный конец Дохнула буря цвет прекрасный Увял утренней заре Потух огонь алтаре XXXII Недвижим лежал странен Был томный мир чела Под грудь навылет ранен Дымясь раны кровь текла Тому одно мгновенье В сем сердце билось вдохновенье Вражда надежда любовь Играла кипела кровь Теперь доме о

In [496]:
# функция потерь

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# подсчитаем ошибку
example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (16, 200, 8911)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       9.095045


Обучение модели

In [497]:
# место для хранения checkpoint
checkpoint_dir = '/content/training_checkpoints'
# Имя файла checkpoint
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=9*30, # сохраняем каждую 30-ю итерацию - в эпохе 44 батча (это видно при запуске обучения)
    save_weights_only=True, # будем сохранять только веса
    )

In [498]:
# Компиляция модели
model.compile(optimizer='adam', loss=loss)

In [499]:
#обучение модели
EPOCHS = 60
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/60
5/5 [==============================] - 8s 311ms/step - loss: 8.9931
Epoch 2/60
5/5 [==============================] - 2s 304ms/step - loss: 8.6836
Epoch 3/60
5/5 [==============================] - 2s 305ms/step - loss: 8.4803
Epoch 4/60
5/5 [==============================] - 2s 306ms/step - loss: 8.4106
Epoch 5/60
5/5 [==============================] - 2s 307ms/step - loss: 8.3915
Epoch 6/60
5/5 [==============================] - 2s 306ms/step - loss: 8.3877
Epoch 7/60
5/5 [==============================] - 2s 312ms/step - loss: 8.3774
Epoch 8/60
5/5 [==============================] - 2s 306ms/step - loss: 8.3822
Epoch 9/60
5/5 [==============================] - 2s 303ms/step - loss: 8.3687
Epoch 10/60
5/5 [==============================] - 2s 304ms/step - loss: 8.3759
Epoch 11/60
5/5 [==============================] - 2s 302ms/step - loss: 8.3687
Epoch 12/60
5/5 [==============================] - 2s 305ms/step - loss: 8.3636
Epoch 13/60
5/5 [==============================] 

In [500]:
#предсказание модели
example_batch_predictions = model(input_example_batch)
print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(16, 200, 8911) # (batch_size, sequence_length, vocab_size)


In [501]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [502]:
print("Input: \n", repr("".join(idx2token[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2token[sampled_indices ])))

Input: 
 'пеньСтановитсяГильосмущенныйПлащибросаютврагаЗарецкийтридцатьшагаОтмерилточностьюотменнойДрузейразвелкрайнийследИвзялсвойпистолетXXXТеперьсходитесьХладнокровноЕщецеляврагаПоходкойтвердойтихоровноЧетыреперешлишагаЧетыресмертныеступениСвойпистолетЕвгенийНепреставаянаступатьСталтихоподыматьВотшаговступилиИЛенскийжмурялевыйглазСталцелитьОнегинвыстрелилПробилиЧасыурочныепоэтРоняетмолчапистолетXXXIНагрудькладеттихонькорукуИпадаетТуманныйвзорИзображаетсмертьмукуТакмедленноскатугорНасолнцеискрамиблистаяСпадаетглыбаснеговаяМгновеннымхолодомоблитОнегинюношеспешитГлядитзоветнапрасноЕгоМладойпевецНашелбезвременныйконецДохнулабуряцветпрекрасныйУвялутреннейзареПотухогоньалтареXXXIIНедвижимлежалстраненБылтомныймирчелаПодгрудьнавылетраненДымясьраныкровьтеклаТомуодномгновеньеВсемсердцебилосьвдохновеньеВражданадеждалюбовьИгралакипелакровьТеперьдомеопустеломВсетихотемноЗамолклонавсегдаЗакрытыставниокнымеломЗабеленыХозяйкиАбогвестьПропалследXXXIIIПриятнодерзкойэпиграммойВзбеситьоплошноговрагаПри

### Генерация текста

In [503]:
#Находим имя файла последней сохраненной контрольной точки
tf.train.latest_checkpoint(checkpoint_dir)

'/content/training_checkpoints/ckpt_54'

In [504]:
#строим модель
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

#загружаем веса из последней сохраненной контрольной точки в модель
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [505]:
#информация о модели
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (1, None, 512)            4562432   
                                                                 
 lstm_51 (LSTM)              (1, None, 1024)           6295552   
                                                                 
 lstm_52 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 lstm_53 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 lstm_54 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 dense_13 (Dense)            (1, None, 8911)           9133775   
                                                                 
Total params: 45,169,871
Trainable params: 45,169,871

In [523]:
def generate_text(model, start_string):
    # Этап оценки (генерация текста с использованием обученной модели)

    # число токенов для генераци
    num_generate = 10

    # Преобразование начальной строки в числа (векторизация)
    input_eval = [token2idx[s] for s in make_tokens(start_string)]
    #Возвращаем тензор с осью длины 1, вставленной первой в индекс
    input_eval = tf.expand_dims(input_eval, 0)
    # print(input_eval)

    # Пустая строка для хранения результатов
    text_generated = []

    # Низкая температура приводит к более предсказуемому тексту.
    # Более высокая температура приводит к более неожиданному тексту.
    temperature = 0.5

    # здесь batch size == 1
    # сбрасываем состояния всех слоев в модели
    model.reset_states()
    for i in range(num_generate):

        #получаем предсказания модели
        predictions = model(input_eval)

        #удаляем первую размерность в предсказании
        predictions = tf.squeeze(predictions, 0)

        # использование категориального распределения для прогнозирования символа, возвращаемого моделью
        # predictions = predictions / temperature

        # выберем последний токен из отсэмплированных предсказаний[-1], т.к. именно он будет предсказанным следующим токеном в строке
        # индекс 0 - выбираем именно индекс токена (помимо него выводится еще размер [1]и тип [2])
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        # print(predicted_id)

        # Передаем предсказанный символ в качестве следующего ввода в модель, по нему будет предсказывать следующий символ
        input_eval = tf.expand_dims([predicted_id], 0)

        #сохраняем предсказанную букву
        text_generated.append(idx2token[predicted_id])

    return (start_string + ' '.join(text_generated))
        

In [524]:
text_ = generate_text(model, start_string=u"Вот он идет ")
print(text_)

Вот он идет Довольно Отец Его Татьяна пробужден лежит Евгений представить благословил холодный
